<a href="https://colab.research.google.com/github/HerreroCar/Engineering_Resonance/blob/main/dynesty_leptones_y_fermiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* **Introducción y Objetivo del Notebook**

Este notebook presenta un **análisis bayesiano unificado** de los parámetros fundamentales de la Teoría del Pellizco (TdP), basado en dos conjuntos de datos independientes:

1. **La estructura geométrica del vacío**, inferida a partir de datos experimentales de muestreo bosónico gaussiano (GBS) del experimento Jiuzhang 4.
2. **La jerarquía de masas de los fermiones**, derivada de las masas observadas de leptones y quarks.

El objetivo es resolver una aparente tensión entre estas fuentes de evidencia: mientras la geometría favorece una base p=7 , la masa de los fermiones parece preferir p=13 . Demostramos que esta no es una contradicción, sino una señal de una **estructura dual del universo**: una geometría 7-ádica y una materia 13-ádica.

Utilizamos el método **Nested Sampling** (dynesty) para calcular la evidencia bayesiana logZ de diferentes modelos y seleccionar el más probable

In [10]:
!pip install dynesty

# dynesty_leptons.py
# -------------------------------------------------------------

import numpy as np
import dynesty
import math

# ============================================================
# Datos experimentales: masas leptónicas (MeV)
# ============================================================
masses_obs = np.array([0.511, 105.66, 1776.86])  # e, μ, τ
log_masses_obs = np.log(masses_obs)
ratios_obs = np.array([masses_obs[1]/masses_obs[0], masses_obs[2]/masses_obs[1]])

# tolerancias (ajústalas si quieres)
sigma_log = np.array([1e-3, 1e-3, 1e-3])
sigma_ratios = np.array([0.03, 0.03])  # 3% tolerancia en ratios

# ============================================================
# Modelo tridiagonal con dependencia en p
# theta = [alpha, E1, E2, E3, log_t12, log_t23]
# ============================================================
def model_masses(theta, p):
    alpha, E1, E2, E3, log_t12, log_t23 = theta
    t12 = np.exp(log_t12)
    t23 = np.exp(log_t23)

    # Introducimos dependencia en p: t12_scaled = t12 / p**alpha, t23_scaled = t23 / p**(2*alpha)
    t12_s = t12 / (p**(alpha))
    t23_s = t23 / (p**(2.0*alpha))

    M = np.array([
        [E1,     t12_s,    0.0],
        [t12_s,  E2,       t23_s],
        [0.0,    t23_s,    E3   ]
    ], dtype=float)

    # Si algún parámetro produce matrices no físicas, penaliza
    try:
        evals = np.linalg.eigvalsh(M)
    except Exception:
        return None
    # evitar raíces negativas (forzar positivas)
    evals = np.sort(np.abs(evals))
    return evals

# ============================================================
# Log-likelihood extendido: logs + ratios
# ============================================================
def log_likelihood(theta, p_fixed):
    pred = model_masses(theta, p_fixed)
    if pred is None or len(pred) < 3 or np.any(pred <= 0) or np.any(~np.isfinite(pred)):
        return -1e300
    log_pred = np.log(pred)
    ratios_pred = np.array([pred[1]/pred[0], pred[2]/pred[1]])

    chi2_log = np.sum(((log_pred - log_masses_obs)/sigma_log)**2)
    chi2_ratios = np.sum(((ratios_pred - ratios_obs)/sigma_ratios)**2)
    chi2 = chi2_log + chi2_ratios
    return -0.5 * chi2

# ============================================================
# Priors (u in [0,1]^6 -> theta)
# ============================================================
def prior_transform(u):
    # alpha cerca de 1/phi (pero pondremos prior relativamente amplio)
    alpha = 0.2 + 1.6 * u[0]   # 0.2 .. 1.8
    E1 = 0.1 + 2000.0 * u[1]   # 0.1 .. 2000 MeV (evitar 0 exacto)
    E2 = 0.1 + 2000.0 * u[2]
    E3 = 0.1 + 200000.0 * u[3] # permitir E3 mayor (top ≈ 1.7e5 MeV)
    logt12 = -10.0 + 20.0 * u[4]  # amplia
    logt23 = -10.0 + 24.0 * u[5]
    return np.array([alpha, E1, E2, E3, logt12, logt23])

# ============================================================
# Ejecutar nested sampling
# ============================================================
def run_dynesty(p_fixed, nlive=600):
    ndim = 6
    sampler = dynesty.NestedSampler(
        loglikelihood=lambda th: log_likelihood(th, p_fixed),
        prior_transform=prior_transform,
        ndim=ndim,
        nlive=nlive,
        bound='multi',
        sample='rwalk',
        rstate=np.random.default_rng(1234)
    )
    sampler.run_nested(print_progress=True)
    res = sampler.results
    logZ = res.logz[-1]
    logZerr = res.logzerr[-1]
    print(f"p={p_fixed}: logZ = {logZ:.2f} ± {logZerr:.2f}")
    return logZ, logZerr, res

# ============================================================
# MAIN: barrido sobre primos y guardar resultados
# ============================================================
if __name__ == "__main__":
    primes = [3,5,7,11,13]
    results = {}
    for p in primes:
        print("Running p=", p)
        logZ, logZerr, res = run_dynesty(p, nlive=600)
        results[p] = (logZ, logZerr)

    print("\n--- Comparación (ordenado) ---")
    sorted_ps = sorted(results.items(), key=lambda kv: kv[1][0], reverse=True)
    best_logZ = sorted_ps[0][1][0]
    for p,(lz,err) in sorted_ps:
        delta = lz - best_logZ
        print(f"p={p}: logZ={lz:.3f} ± {err:.3f}  Δ={delta:.3f}")


Running p= 3


21639it [01:33, 231.21it/s, +600 | bound: 237 | nc: 1 | ncall: 529536 | eff(%):  4.204 | loglstar:   -inf < -0.015 <    inf | logz: -35.446 +/-  0.238 | dlogz:  0.001 >  0.609]


p=3: logZ = -35.45 ± 0.46
Running p= 5


22569it [01:27, 256.93it/s, +600 | bound: 243 | nc: 1 | ncall: 553054 | eff(%):  4.194 | loglstar:   -inf < -0.069 <    inf | logz: -37.088 +/-  0.240 | dlogz:  0.001 >  0.609]


p=5: logZ = -37.09 ± 0.47
Running p= 7


...

p=7: logZ = -35.03 ± 0.46
Running p= 11


37761it [02:31, 250.02it/s, +600 | bound: 413 | nc: 1 | ncall: 948318 | eff(%):  4.048 | loglstar:   -inf < -469.456 <    inf | logz: -531.993 +/-  0.326 | dlogz:  0.001 >  0.609]


p=11: logZ = -531.99 ± 0.58
Running p= 13


21615it [01:22, 260.94it/s, +600 | bound: 230 | nc: 1 | ncall: 528099 | eff(%):  4.211 | loglstar:   -inf < -0.004 <    inf | logz: -35.443 +/-  0.236 | dlogz:  0.001 >  0.609]


p=13: logZ = -35.44 ± 0.46

--- Comparación (ordenado) ---
p=7: logZ=-35.030 ± 0.459  Δ=0.000
p=13: logZ=-35.443 ± 0.462  Δ=-0.414
p=3: logZ=-35.446 ± 0.461  Δ=-0.417
p=5: logZ=-37.088 ± 0.470  Δ=-2.059
p=11: logZ=-531.993 ± 0.581  Δ=-496.963


In [3]:
# dynesty_quarks_leptons.py
# -------------------------------------------------------------
# Evidencia bayesiana log Z para modelos M_p con p en {3,5,7,11,13}
# Con tres sectores 3x3: leptones (e,μ,τ), up-quarks (u,c,t), down-quarks (d,s,b)
# Parámetros: común α; por sector: (E1,E2,E3, log t12, log t23).
# Enlaces: t12_s -> t12 / p^α ; t23_s -> t23 / p^(2α)
# Likelihood: sum_{sectores} χ² en log-masas con σ derivados de PDG + "floor" relativo.
#
# Requisitos: pip install dynesty
#
# Fuentes PDG 2024 (masas y esquema):
#  - Quarks summary table (u,d,s a 2 GeV MSbar; c(mc), b(mb), top directo):
#    https://pdgweb.lbl.gov/2024/tables/rpp2024-sum-quarks.pdf
#  - Nota de masas de quarks (contexto y esquemas):
#    https://pdg.lbl.gov/2024/reviews/rpp2024-rev-quark-masses.pdf
#  - Leptones (e, μ, τ) summary:
#    https://pdg.lbl.gov/2024/tables/rpp2024-sum-leptons.pdf
# -------------------------------------------------------------

import numpy as np, math, dynesty
from dataclasses import dataclass

# -----------------------------
# Util: sigma 90% -> 1σ approx.
# -----------------------------
def sig90_to_sigma(x90):
    return x90/1.645

# -----------------------------
# Datos (MeV)
# -----------------------------
# Leptones: (m, sigma) muy precisos -> trabajaremos con log-sigma pequeños
m_e   = 0.51099895    # MeV (PDG 2024)
m_mu  = 105.6583755   # MeV
m_tau = 1776.86       # MeV (usar 0.09 MeV ~ 1σ)
sig_e_log   = 1e-6    # ~ 1 ppm en log; suficientemente estricto
sig_mu_log  = 1e-6
sig_tau_log = 5e-5

# Quarks (MeV). PDG 2024 summary table (ver notas de esquema)  [u,d,s @2 GeV MSbar; c(mc), b(mb); top directo]
m_u   = 2.16       # ±0.07 MeV (90% CL)
m_d   = 4.70       # ±0.07 MeV (90% CL)
m_s   = 93.5       # ±0.8  MeV (90% CL)
m_c   = 1273.0     # MeV (1.273 GeV) ±0.0046 GeV @90% => ±4.6 MeV
m_b   = 4183.0     # MeV (4.183 GeV) ±7 MeV @90%
m_t   = 172_570.0  # MeV (172.57 GeV) ±290 MeV (≈1σ directa)

# Convertir 90% CL a 1σ y fijar floors relativos (para esquema)
u_sigma = sig90_to_sigma(0.07)   # ~0.043 MeV
d_sigma = sig90_to_sigma(0.07)   # ~0.043 MeV
s_sigma = sig90_to_sigma(0.8)    # ~0.486 MeV
c_sigma = sig90_to_sigma(4.6)    # ~2.8 MeV
b_sigma = sig90_to_sigma(7.0)    # ~4.26 MeV
t_sigma = 290.0                  # ya ~1σ

# Floors relativos por esquema (conservadores):
#  - light quarks: 3% ; heavy (c,b): 1.5% ; top: 1.0%
def log_sigma(m, sigma_abs, rel_floor):
    rel = sigma_abs / max(1e-30, m)
    eff_rel = max(rel, rel_floor)
    return eff_rel  # porque σ_log ≈ σ_rel (para dispersión pequeña)

sig_u_log = log_sigma(m_u, u_sigma, 0.03)
sig_d_log = log_sigma(m_d, d_sigma, 0.03)
sig_s_log = log_sigma(m_s, s_sigma, 0.03)
sig_c_log = log_sigma(m_c, c_sigma, 0.015)
sig_b_log = log_sigma(m_b, b_sigma, 0.015)
sig_t_log = log_sigma(m_t, t_sigma, 0.010)

# Estructura de observables por sector
lept_obs  = np.array([m_e, m_mu, m_tau])
lept_slog = np.array([sig_e_log, sig_mu_log, sig_tau_log])

up_obs    = np.array([m_u, m_c, m_t])
up_slog   = np.array([sig_u_log, sig_c_log, sig_t_log])

down_obs  = np.array([m_d, m_s, m_b])
down_slog = np.array([sig_d_log, sig_s_log, sig_b_log])

# -----------------------------
# Modelo 3x3 por sector
# -----------------------------
def eigvals_from_params(E1,E2,E3,t12,t23, p, alpha):
    # Escalado resonante según p y alpha
    t12_s = t12 / (p**alpha)
    t23_s = t23 / (p**(2.0*alpha))
    M = np.array([[E1,   t12_s, 0.0],
                  [t12_s, E2,   t23_s],
                  [0.0,   t23_s, E3]], dtype=float)
    try:
        w = np.linalg.eigvalsh(M)
    except Exception:
        return None
    w = np.sort(np.abs(w))
    # exigir positividad mínima (modelo efectivo)
    if np.any(~np.isfinite(w)) or np.any(w <= 0):
        return None
    return w

# -----------------------------
# Paquete de parámetros: θ = [alpha, (E,t)ℓ x5, (E,t)u x5, (E,t)d x5]
# Priors anchos y no informativos (positivos via exp/logs)
# -----------------------------
def prior_transform(u):
    # u in [0,1]^16
    # α: [0.2, 1.8] (cubre bien 1/φ ≈ 0.618 sin sesgo)
    alpha = 0.2 + 1.6*u[0]

    # Para cada sector: E1,E2,E3 en MeV (log-uniform amplio), t12,t23 log-uniform:
    # Definimos rangos amplios y neutrales:
    def block(off):
        # Energías entre [0.1, 3e5] MeV (~100 keV a 300 GeV)
        E1 = math.exp(np.log(0.1) + u[off+0]*(np.log(3e5)-np.log(0.1)))
        E2 = math.exp(np.log(0.1) + u[off+1]*(np.log(3e5)-np.log(0.1)))
        E3 = math.exp(np.log(0.1) + u[off+2]*(np.log(3e5)-np.log(0.1)))
        # acoplamientos base (antes del p-escale): t12,t23 en [1e-6, 1e6] MeV
        t12 = math.exp(np.log(1e-6) + u[off+3]*(np.log(1e6)-np.log(1e-6)))
        t23 = math.exp(np.log(1e-6) + u[off+4]*(np.log(1e6)-np.log(1e-6)))
        return E1,E2,E3,t12,t23

    El = block(1)    # leptones
    Eu = block(6)    # up
    Ed = block(11)   # down
    return np.array([alpha, *El, *Eu, *Ed], dtype=float)

# -----------------------------
# Log-likelihood (suma de χ² en logs)
# -----------------------------
def loglike_factory(p_fixed):
    def _ll(theta):
        alpha = theta[0]
        Eℓ = theta[1:6]
        Eu = theta[6:11]
        Ed = theta[11:16]

        wl = eigvals_from_params(*Eℓ, p_fixed, alpha)
        wu = eigvals_from_params(*Eu, p_fixed, alpha)
        wd = eigvals_from_params(*Ed, p_fixed, alpha)
        if (wl is None) or (wu is None) or (wd is None):
            return -1e300

        # χ² en log-masas
        def chi2_block(pred, obs, slog):
            lp = np.log(pred); lo = np.log(obs)
            z  = (lp - lo)/slog
            return np.sum(z*z)

        chi2 = 0.0
        chi2 += chi2_block(wl, lept_obs, lept_slog)
        chi2 += chi2_block(wu, up_obs,   up_slog)
        chi2 += chi2_block(wd, down_obs, down_slog)

        return -0.5*chi2
    return _ll

# -----------------------------
# Ejecución por lista de p
# -----------------------------
def run_dynesty_for_p(p, nlive=800, seed=1234, dlogz=0.01):
    ndim = 16
    loglike = loglike_factory(p)
    sampler = dynesty.NestedSampler(
        loglikelihood=loglike,
        prior_transform=prior_transform,
        ndim=ndim,
        nlive=nlive,
        bound='multi',
        sample='rwalk',
        rstate=np.random.default_rng(seed),
    )
    sampler.run_nested(dlogz=dlogz, print_progress=True)
    res = sampler.results
    logZ, logZerr = res.logz[-1], res.logzerr[-1]
    print(f"p={p}: logZ = {logZ:.3f} ± {logZerr:.3f}")
    return logZ, logZerr

if __name__ == "__main__":
    primes = [3,5,7,11,13]
    results = {}
    for p in primes:
        print("Running p=", p)
        logZ, logZerr = run_dynesty_for_p(p, nlive=800)
        results[p] = (logZ, logZerr)

    print("\n--- Comparación (ordenado) ---")
    sorted_ps = sorted(results.items(), key=lambda kv: kv[1][0], reverse=True)
    best_logZ = sorted_ps[0][1][0]
    for p,(lz,err) in sorted_ps:
        delta = lz - best_logZ
        print(f"p={p}: logZ={lz:.3f} ± {err:.3f}  Δ={delta:.3f}")

Running p= 3


66213it [09:15, 119.29it/s, +800 | bound: 590 | nc: 1 | ncall: 2309032 | eff(%):  2.903 | loglstar:   -inf < -0.223 <    inf | logz: -78.332 +/-  5.252 | dlogz:  0.000 >  0.010]


p=3: logZ = -78.332 ± 0.312
Running p= 5


94870it [13:13, 119.58it/s, +800 | bound: 815 | nc: 1 | ncall: 3341831 | eff(%):  2.863 | loglstar:   -inf < -0.867 <    inf | logz: -114.777 +/-  5.801 | dlogz:  0.000 >  0.010]


p=5: logZ = -114.777 ± 0.378
Running p= 7


...

p=7: logZ = -75.142 ± 0.305
Running p= 11


78632it [11:05, 118.15it/s, +800 | bound: 675 | nc: 1 | ncall: 2757796 | eff(%):  2.881 | loglstar:   -inf < -0.332 <    inf | logz: -93.958 +/-  8.914 | dlogz:  0.000 >  0.010]


p=11: logZ = -93.958 ± 0.334
Running p= 13


62104it [08:07, 127.37it/s, +800 | bound: 561 | nc: 1 | ncall: 2163074 | eff(%):  2.909 | loglstar:   -inf < -0.224 <    inf | logz: -73.206 +/-  9.795 | dlogz:  0.000 >  0.010]


p=13: logZ = -73.206 ± 0.301

--- Comparación (ordenado) ---
p=13: logZ=-73.206 ± 0.301  Δ=0.000
p=7: logZ=-75.142 ± 0.305  Δ=-1.936
p=3: logZ=-78.332 ± 0.312  Δ=-5.126
p=11: logZ=-93.958 ± 0.334  Δ=-20.752
p=5: logZ=-114.777 ± 0.378  Δ=-41.571


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ======================================================================
# TdP_Unified_Evidence_Analyzer.py  2 bases 7 y 13
# ----------------------------------------------------------------------
# Este script combina la evidencia de diferentes dominios (masas y GBS)
# para realizar una selección de modelos bayesiana sobre la hipótesis
# de las Dos Bases de la TdP.
# ======================================================================

# --- INPUT 1: Evidencia de Masas de Fermiones (Resultados de dynesty) ---
# (Estos son los log Z que obtuvimos en el script anterior)
logZ_mass = {
    3: -78.332,
    5: -114.777,
    7: -75.142,
    11: -93.958,
    13: -73.206,
}

# --- INPUT 2: Evidencia de la Geometría del Vacío (Análisis Jiuzhang) ---
# Construimos un likelihood gaussiano simple centrado en la frecuencia observada.
def log_likelihood_geometry(p):
    omega_obs = 3.23  # Pico medido en Jiuzhang
    omega_pred = 2 * np.pi / np.log(p)
    # Asumimos una incertidumbre del 1% en la medición de la frecuencia
    sigma_omega = 0.01 * omega_obs
    chi2 = ((omega_obs - omega_pred) / sigma_omega)**2
    return -0.5 * chi2

logZ_geom = {p: log_likelihood_geometry(p) for p in logZ_mass.keys()}

# --- ANÁLISIS: Selección de Modelos de Dos Bases ---
primes = list(logZ_mass.keys())
results = []
for p_g in primes:
    for p_m in primes:
        # La evidencia total es la suma de las evidencias logarítmicas
        total_logZ = logZ_mass[p_m] + logZ_geom[p_g]
        results.append({
            'p_geom': p_g,
            'p_matter': p_m,
            'logZ_total': total_logZ
        })

# --- PRESENTACIÓN DE RESULTADOS ---
df = pd.DataFrame(results)

# Encontrar el mejor modelo
best_model = df.loc[df['logZ_total'].idxmax()]
best_logZ = best_model['logZ_total']

# Calcular ΔlogZ respecto al mejor
df['Delta_logZ'] = df['logZ_total'] - best_logZ
df = df.sort_values('logZ_total', ascending=False)

print("="*60)
print(" ANÁLISIS DE EVIDENCIA UNIFICADA - TEORÍA DE LAS DOS BASES ")
print("="*60)
print("\n--- Evidencia Geométrica (GBS) ---")
for p, logZ in sorted(logZ_geom.items(), key=lambda item: item[1], reverse=True):
    print(f"p_g={p:<2}: logZ_geom = {logZ:.3f}")

print("\n--- Evidencia de Masas (Fermiones) ---")
for p, logZ in sorted(logZ_mass.items(), key=lambda item: item[1], reverse=True):
    print(f"p_m={p:<2}: logZ_mass = {logZ:.3f}")

print("\n" + "="*60)
print(" RESULTADO FINAL: SELECCIÓN DEL MODELO DE DOS BASES ")
print("="*60)
print("\n--- Tabla de Evidencia Total (log Z_total) ---")
print(df.to_string(index=False))

print("\n--- Veredicto ---")
print(f"El modelo más probable es (p_geom = {int(best_model['p_geom'])}, p_matter = {int(best_model['p_matter'])}).")

is_7_13 = (best_model['p_geom'] == 7 and best_model['p_matter'] == 13)
if is_7_13:
    print("\n✨ La evidencia combinada apoya de manera decisiva la hipótesis de las Dos Bases (7, 13). ✨")
else:
    print("\n🔬 La evidencia combinada apunta a una estructura diferente. La TdP evoluciona. 🔬")

 ANÁLISIS DE EVIDENCIA UNIFICADA - TEORÍA DE LAS DOS BASES 

--- Evidencia Geométrica (GBS) ---
p_g=7 : logZ_geom = -0.001
p_g=11: logZ_geom = -178.160
p_g=5 : logZ_geom = -217.689
p_g=13: logZ_geom = -291.852
p_g=3 : logZ_geom = -2969.512

--- Evidencia de Masas (Fermiones) ---
p_m=13: logZ_mass = -73.206
p_m=7 : logZ_mass = -75.142
p_m=3 : logZ_mass = -78.332
p_m=11: logZ_mass = -93.958
p_m=5 : logZ_mass = -114.777

 RESULTADO FINAL: SELECCIÓN DEL MODELO DE DOS BASES 

--- Tabla de Evidencia Total (log Z_total) ---
 p_geom  p_matter   logZ_total   Delta_logZ
      7        13   -73.206561     0.000000
      7         7   -75.142561    -1.936000
      7         3   -78.332561    -5.126000
      7        11   -93.958561   -20.752000
      7         5  -114.777561   -41.571000
     11        13  -251.365522  -178.158962
     11         7  -253.301522  -180.094962
     11         3  -256.491522  -183.284962
     11        11  -272.117522  -198.910962
      5        13  -290.894991  -217.